### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "OfficeHome"
MODEL_NAME = "resnet18"

c:\Users\Fatim_Sproj\anaconda3\envs\bacp\lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-11-15 00:25:07,600 | INFO | Initialized experiment directories for OfficeHome
2025-11-15 00:25:07,600 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet18_experiments\OfficeHome\logs
2025-11-15 00:25:07,600 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet18_experiments\OfficeHome\checkpoints
2025-11-15 00:25:07,600 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet18_experiments\OfficeHome\plots


### Setup

In [4]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]

{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [5]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

2025-11-14 23:52:03,103 | INFO | === LODO: Leaving out domain 'Art' ===



=== LODO: Leaving out domain 'Art' ===


Evaluating: 100%|██████████| 19/19 [00:11<00:00,  1.62it/s]
2025-11-14 23:52:39,702 | INFO | [Art] Epoch 1/5 | Train - Loss: 2.4474, Cls: 2.4389, GRQO: 0.0085, Acc: 0.4694 | Val - Loss: 1.9892, Cls: 1.9865, GRQO: 0.0027, Acc: 0.5175
2025-11-14 23:52:39,778 | INFO | [Art] New best val acc: 0.5175


[Art] Epoch 1/5 | Train - Loss: 2.4474, Cls: 2.4389, GRQO: 0.0085, Acc: 0.4694 | Val - Loss: 1.9892, Cls: 1.9865, GRQO: 0.0027, Acc: 0.5175
[Art] New best val acc: 0.5175


Evaluating: 100%|██████████| 19/19 [00:11<00:00,  1.63it/s]
2025-11-14 23:53:15,902 | INFO | [Art] Epoch 2/5 | Train - Loss: 0.7542, Cls: 0.7499, GRQO: 0.0043, Acc: 0.8233 | Val - Loss: 1.8263, Cls: 1.8248, GRQO: 0.0015, Acc: 0.5492
2025-11-14 23:53:15,970 | INFO | [Art] New best val acc: 0.5492


...

[Real World] Epoch 3/5 | Train - Loss: 0.3639, Cls: 0.3603, GRQO: 0.0036, Acc: 0.9251 | Val - Loss: 1.0764, Cls: 1.0757, GRQO: 0.0008, Acc: 0.7106
[Real World] New best val acc: 0.7106


Evaluating: 100%|██████████| 35/35 [00:56<00:00,  1.61s/it]
2025-11-15 00:08:57,421 | INFO | [Real World] Epoch 4/5 | Train - Loss: 0.1466, Cls: 0.1435, GRQO: 0.0031, Acc: 0.9746 | Val - Loss: 1.0552, Cls: 1.0545, GRQO: 0.0006, Acc: 0.7253
2025-11-15 00:08:57,532 | INFO | [Real World] New best val acc: 0.7253


[Real World] Epoch 4/5 | Train - Loss: 0.1466, Cls: 0.1435, GRQO: 0.0031, Acc: 0.9746 | Val - Loss: 1.0552, Cls: 1.0545, GRQO: 0.0006, Acc: 0.7253
[Real World] New best val acc: 0.7253


Evaluating: 100%|██████████| 35/35 [00:49<00:00,  1.41s/it]
2025-11-15 00:10:11,736 | INFO | [Real World] Epoch 5/5 | Train - Loss: 0.0768, Cls: 0.0741, GRQO: 0.0027, Acc: 0.9866 | Val - Loss: 1.0650, Cls: 1.0646, GRQO: 0.0005, Acc: 0.7283
2025-11-15 00:10:11,806 | INFO | [Real World] New best val acc: 0.7283
2025-11-15 00:10:11,807 | INFO | [Real World] Best Acc: 0.7283
2025-11-15 00:10:11,807 | INFO | ------------------------------------------------------------
2025-11-15 00:10:11,809 | INFO | LODO finished | Mean Acc: 0.6226 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet18_experiments\OfficeHome\logs\lodo_summary_20251115_001011.json


[Real World] Epoch 5/5 | Train - Loss: 0.0768, Cls: 0.0741, GRQO: 0.0027, Acc: 0.9866 | Val - Loss: 1.0650, Cls: 1.0646, GRQO: 0.0005, Acc: 0.7283
[Real World] New best val acc: 0.7283
[Real World] Best Acc: 0.7283
------------------------------------------------------------
LODO finished | Mean Acc: 0.6226
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet18_experiments\OfficeHome\logs\lodo_summary_20251115_001011.json


### Baseline

In [ ]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)

2025-11-15 00:25:07,866 | INFO | Initializing ResNet baseline: resnet18


Initializing ResNet baseline: resnet18


2025-11-15 00:25:08,162 | INFO | === Baseline LODO: Leaving out domain 'Art' ===



=== Baseline LODO: Leaving out domain 'Art' ===


2025-11-15 00:25:53,599 | INFO | [Art] Epoch 1/5 | Train - Loss: 2.2337, Acc: 0.5308 | Val Acc: 0.4841


...

[Product] Epoch 3/5 | Train - Loss: 0.5843, Acc: 0.8951 | Val Acc: 0.7292


2025-11-15 00:33:50,547 | INFO | [Product] Epoch 4/5 | Train - Loss: 0.3074, Acc: 0.9542 | Val Acc: 0.7297


[Product] Epoch 4/5 | Train - Loss: 0.3074, Acc: 0.9542 | Val Acc: 0.7297


2025-11-15 00:34:27,512 | INFO | [Product] Epoch 5/5 | Train - Loss: 0.1663, Acc: 0.9803 | Val Acc: 0.7330
2025-11-15 00:34:27,513 | INFO | [Product] Best Val Acc: 0.7330
2025-11-15 00:34:27,513 | INFO | ------------------------------------------------------------
2025-11-15 00:34:27,513 | INFO | Initializing ResNet baseline: resnet18
2025-11-15 00:34:27,595 | INFO | === Baseline LODO: Leaving out domain 'Real World' ===


[Product] Epoch 5/5 | Train - Loss: 0.1663, Acc: 0.9803 | Val Acc: 0.7330
[Product] Best Val Acc: 0.7330
------------------------------------------------------------
Initializing ResNet baseline: resnet18

=== Baseline LODO: Leaving out domain 'Real World' ===
